In [1]:
import pandas as pd
import numpy as np
import os
import datetime

In [2]:
### Run Time Function
def print_runtime(t1, t2):
    tot_sec = t2.timestamp()-t1.timestamp()
    hours = tot_sec//3600
    minutes = (tot_sec-hours*3600)//60
    seconds = tot_sec-hours*3600-minutes*60

    print("Run Time:", hours, 'hrs', minutes, 'mins', round(seconds), "sec")
    
    return

In [3]:
### Directory of the simulation.py file
simpy_dir = r'C:/abm_runs/rohans'

### Location of Configs
settings = r'C:/abm_runs/rohans/configs/resident/settings_mp.yaml'
configs_resident = r'C:/abm_runs/rohans/configs/resident'
configs_common = r'C:/abm_runs/rohans/configs/common'

### Location of auto_ownership_coefficients
ao_coef_loc = r'C:/abm_runs/rohans/configs/resident'

### Location of AO targets
base_target_loc = r'C:/abm_runs/rohans/ao_calibration/targets/ao_targets_2021.csv'

### Location of Input and Output data
data = r'C:/abm_runs/rohans/input'
output_dir = r'C:/abm_runs/rohans/output'

In [4]:
### Define dampening or enhancing factor
damp_fac = 1.5

### Read in base targets/frequency
base_share = pd.read_csv(base_target_loc, index_col=0)
base_share

,share
auto_ownership,
0,0.054026
1,0.295868
2,0.398939
3,0.160256
4,0.090911


### Cold Start: 
Run ActivitySim up to joint tour composition model by editing settings.yaml

In [19]:
i = 0
### Read and backup existing auto_ownership_coefficients
ao_coef = pd.read_csv(ao_coef_loc+'/auto_ownership_coefficients.csv')
ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients_'+str(i)+'.csv', index=False)

In [5]:
start_time = datetime.datetime.now()

#Run ActivitySim
os.chdir(simpy_dir)
asim_command ='python simulation.py -s ' + settings + ' -c ' + configs_resident + ' -c ' + configs_common + ' -d ' + data + ' -o ' + output_dir

os.system(asim_command)

0

In [6]:
end_time = datetime.datetime.now()
print_runtime(start_time, end_time)

Run Time: 1.0 hrs 30.0 mins 36 sec


In [22]:
### Read households files
hh = pd.read_csv(output_dir + "/final_households.csv")
hh['weights'] = 1/hh['sample_rate']

### Calculate asim shares
asim_freq = pd.pivot_table(hh, index='auto_ownership', values='weights', aggfunc=np.sum)
asim_share = asim_freq/asim_freq.sum()
# asim_share.reset_index(inplace=True)

### Calculate calibration constants
ao_calib = base_share.rename({'share': 'census'}, axis=1)
ao_calib['model'] = asim_share['weights']
ao_calib['ratio'] = ao_calib['census']/ao_calib['model']
ao_calib['constants'] = np.log(ao_calib['ratio'])

ao_calib.to_csv('C:/abm_runs/rohans/ao_calibration/ao_calib_summary_'+str(i)+'.csv')
ao_calib

,census,model,ratio,constants
auto_ownership,,,,
0,0.054026,0.134283,0.402328,-0.910487
1,0.295868,0.300773,0.983691,-0.016444
2,0.398939,0.354860,1.124215,0.117085
3,0.160256,0.131713,1.216703,0.196145
4,0.090911,0.078370,1.160023,0.148440


In [23]:
# i=1
# ### Update auto ownership constants
# cali_fac = ao_calib['constants'].drop({1}, axis=0)*damp_fac
# ao_coef.loc[97:, 'value'] = np.array(ao_coef.loc[97:, 'value']) + cali_fac.values

# ### Save auto ownership constants
# ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients_'+str(i)+'.csv', index=False)
# ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients.csv', index=False)

### Warm Start
Edit settings.yaml to resume after and go to write tables

In [24]:
for i in range(1, 2):
    print('Iteration -', i)
    
    ### Update auto ownership constants
    cali_fac = ao_calib['constants'].drop({1}, axis=0)*damp_fac
    ao_coef.loc[98:, 'value'] = np.array(ao_coef.loc[98:, 'value']) + cali_fac.values

    ### Save auto ownership constants
    ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients_'+str(i)+'.csv', index=False)
    ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients.csv', index=False)

    ### Run ActivitySim
    start_time = datetime.datetime.now()
    
    os.chdir(simpy_dir)
    asim_command ='python simulation.py -s ' + settings + ' -c ' + configs_resident + ' -c ' + configs_common + ' -d ' + data + ' -o ' + output_dir
    os.system(asim_command)

    end_time = datetime.datetime.now()
    print_runtime(start_time, end_time)
    
    ### Read households files
    hh = pd.read_csv(output_dir + "/final_households.csv")
    hh['weights'] = 1/hh['sample_rate']

    ### Calculate asim shares
    asim_freq = pd.pivot_table(hh, index='auto_ownership', values='weights', aggfunc=np.sum)
    asim_share = asim_freq/asim_freq.sum()
    asim_share.reset_index(inplace=True)

    ### Calculate calibration constants
    ao_calib = base_share.rename({'share': 'census'}, axis=1)
    ao_calib['model'] = asim_share['weights']
    ao_calib['ratio'] = ao_calib['census']/ao_calib['model']
    ao_calib['constants'] = np.log(ao_calib['ratio'])

    ao_calib.to_csv('C:/abm_runs/rohans/ao_calibration/ao_calib_summary_'+str(i)+'.csv')
    print(ao_calib, '\n')

Iteration - 1
Run Time: 0.0 hrs 31.0 mins 14 sec
                  census     model     ratio  constants
auto_ownership                                         
0               0.054026  0.077490  0.697200  -0.360683
1               0.295868  0.325457  0.909086  -0.095316
2               0.398939  0.369540  1.079556   0.076550
3               0.160256  0.147270  1.088178   0.084505
4               0.090911  0.080243  1.132941   0.124817 



In [25]:
for i in range(2, 5):
    print('Iteration -', i)
    
    ### Update auto ownership constants
    cali_fac = ao_calib['constants'].drop({1}, axis=0)*damp_fac
    ao_coef.loc[98:, 'value'] = np.array(ao_coef.loc[98:, 'value']) + cali_fac.values

    ### Save auto ownership constants
    ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients_'+str(i)+'.csv', index=False)
    ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients.csv', index=False)

    ### Run ActivitySim
    start_time = datetime.datetime.now()
    
    os.chdir(simpy_dir)
    asim_command ='python simulation.py -s ' + settings + ' -c ' + configs_resident + ' -c ' + configs_common + ' -d ' + data + ' -o ' + output_dir
    os.system(asim_command)

    end_time = datetime.datetime.now()
    print_runtime(start_time, end_time)
    
    ### Read households files
    hh = pd.read_csv(output_dir + "/final_households.csv")
    hh['weights'] = 1/hh['sample_rate']

    ### Calculate asim shares
    asim_freq = pd.pivot_table(hh, index='auto_ownership', values='weights', aggfunc=np.sum)
    asim_share = asim_freq/asim_freq.sum()
    asim_share.reset_index(inplace=True)

    ### Calculate calibration constants
    ao_calib = base_share.rename({'share': 'census'}, axis=1)
    ao_calib['model'] = asim_share['weights']
    ao_calib['ratio'] = ao_calib['census']/ao_calib['model']
    ao_calib['constants'] = np.log(ao_calib['ratio'])

    ao_calib.to_csv('C:/abm_runs/rohans/ao_calibration/ao_calib_summary_'+str(i)+'.csv')
    print(ao_calib, '\n')

Iteration - 2
Run Time: 0.0 hrs 31.0 mins 0 sec
                  census     model     ratio  constants
auto_ownership                                         
0               0.054026  0.061243  0.882153  -0.125390
1               0.295868  0.327013  0.904758  -0.100088
2               0.398939  0.377610  1.056484   0.054947
3               0.160256  0.149083  1.074942   0.072267
4               0.090911  0.085050  1.068912   0.066642 

Iteration - 3
Run Time: 0.0 hrs 30.0 mins 54 sec
                  census     model     ratio  constants
auto_ownership                                         
0               0.054026  0.056067  0.963603  -0.037076
1               0.295868  0.322543  0.917297  -0.086324
2               0.398939  0.382630  1.042623   0.041740
3               0.160256  0.152773  1.048979   0.047817
4               0.090911  0.085987  1.057269   0.055689 

Iteration - 4
Run Time: 0.0 hrs 30.0 mins 53 sec
                  census     model     ratio  constants
auto_owner

In [26]:
for i in range(5, 8):
    print('Iteration -', i)
    
    ### Update auto ownership constants
    cali_fac = ao_calib['constants'].drop({1}, axis=0)*damp_fac
    ao_coef.loc[98:, 'value'] = np.array(ao_coef.loc[98:, 'value']) + cali_fac.values

    ### Save auto ownership constants
    ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients_'+str(i)+'.csv', index=False)
    ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients.csv', index=False)

    ### Run ActivitySim
    start_time = datetime.datetime.now()
    
    os.chdir(simpy_dir)
    asim_command ='python simulation.py -s ' + settings + ' -c ' + configs_resident + ' -c ' + configs_common + ' -d ' + data + ' -o ' + output_dir
    os.system(asim_command)

    end_time = datetime.datetime.now()
    print_runtime(start_time, end_time)
    
    ### Read households files
    hh = pd.read_csv(output_dir + "/final_households.csv")
    hh['weights'] = 1/hh['sample_rate']

    ### Calculate asim shares
    asim_freq = pd.pivot_table(hh, index='auto_ownership', values='weights', aggfunc=np.sum)
    asim_share = asim_freq/asim_freq.sum()
    asim_share.reset_index(inplace=True)

    ### Calculate calibration constants
    ao_calib = base_share.rename({'share': 'census'}, axis=1)
    ao_calib['model'] = asim_share['weights']
    ao_calib['ratio'] = ao_calib['census']/ao_calib['model']
    ao_calib['constants'] = np.log(ao_calib['ratio'])

    ao_calib.to_csv('C:/abm_runs/rohans/ao_calibration/ao_calib_summary_'+str(i)+'.csv')
    print(ao_calib, '\n')

Iteration - 5
Run Time: 0.0 hrs 31.0 mins 5 sec
                  census     model     ratio  constants
auto_ownership                                         
0               0.054026  0.054047  0.999618  -0.000382
1               0.295868  0.312520  0.946717  -0.054755
2               0.398939  0.389383  1.024541   0.024244
3               0.160256  0.155880  1.028073   0.027686
4               0.090911  0.088170  1.031088   0.030614 

Iteration - 6
Run Time: 0.0 hrs 31.0 mins 17 sec
                  census     model     ratio  constants
auto_ownership                                         
0               0.054026  0.053960  1.001223   0.001222
1               0.295868  0.308920  0.957750  -0.043169
2               0.398939  0.391377  1.019322   0.019138
3               0.160256  0.156833  1.021824   0.021589
4               0.090911  0.088910  1.022506   0.022256 

Iteration - 7
Run Time: 0.0 hrs 31.0 mins 51 sec
                  census     model     ratio  constants
auto_owner

### Testing

In [ ]:
# i = 1
# print('Iteration -', i)
    
# ### Update auto ownership constants
# cali_fac = ao_calib['constants'].drop({1}, axis=0)*damp_fac
# ao_coef.loc[98:, 'value'] = np.array(ao_coef.loc[98:, 'value']) + cali_fac.values

In [ ]:
# ### Save auto ownership constants
# ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients_'+str(i)+'.csv', index=False)
# ao_coef.to_csv(ao_coef_loc+'/auto_ownership_coefficients.csv', index=False)

In [13]:
# ### Run ActivitySim
# start_time = datetime.datetime.now()

# os.chdir(simpy_dir)
# asim_command ='python simulation.py -s ' + settings + ' -c ' + configs_resident + ' -c ' + configs_common + ' -d ' + data + ' -o ' + output_dir
# os.system(asim_command)

# end_time = datetime.datetime.now()
# print_runtime(start_time, end_time)

Run Time: 0.0 hrs 32.0 mins 25 sec


In [14]:
# ### Read households files
# hh = pd.read_csv(output_dir + "/final_households.csv")
# hh['weights'] = 1/hh['sample_rate']

# ### Calculate asim shares
# asim_freq = pd.pivot_table(hh, index='auto_ownership', values='weights', aggfunc=np.sum)
# asim_share = asim_freq/asim_freq.sum()
# asim_share.reset_index(inplace=True)
# asim_share

,auto_ownership,weights
0,1,0.398707
1,2,0.372920
2,3,0.147883
3,4,0.080490


In [ ]:
# ### Calculate calibration constants
# ao_calib = base_share.rename({'share': 'census'}, axis=1)
# ao_calib['model'] = asim_share['weights']
# ao_calib['ratio'] = ao_calib['census']/ao_calib['model']
# ao_calib['constants'] = np.log(ao_calib['ratio'])

# ao_calib.to_csv('C:/abm_runs/rohans/ao_calibration/ao_calib_summary_'+str(i)+'.csv')
# print(ao_calib, '\n')